In [ ]:
import os
from datetime import time, date, datetime, timedelta
from IPython.display import display, HTML
import polars as pl

os.environ["POLARS_FMT_MAX_ROWS"] = "50"
os.environ["POLARS_FMT_MAX_COLS"] = "50"
os.environ["POLARS_FMT_COL_MAX_WIDTH"] = "50"
pl.Config(fmt_str_lengths=50)

def print_df(df: pl.DataFrame, title=None):
	html = df._repr_html_()
	if title:
		html = f"<h3>{title}</h3>" + html
	display(HTML(html))

now = datetime.now()

# import httpx
# from src.user_list import UserList
# async with httpx.AsyncClient() as client:
# 	user_list = await UserList.from_web("cosmicwaves", client)
# 	user_list.df.sink_parquet("data/user_list.parquet")

user_list = pl.read_parquet("data/user_list.parquet")
anime_db = pl.scan_parquet("data/anime.parquet")

common_cols = set(user_list.columns).intersection(set(anime_db.columns))
if len(common_cols) > 1:
	raise Exception(f"Common columns found between user_list and anime_db: {common_cols}")

user_animes = user_list.lazy().join(anime_db, on="anime_id", how="inner", validate="1:1").collect()

if user_animes.height < user_list.height:
	print(f"Warning: {user_list.height - user_animes.height} anime not found in database")

print_df(user_animes.head(10), "User Animes")

for col in ["user_watch_status", "user_priority", "user_storage", "user_added_to_list", "user_rewatching", "user_scored"]:
	print_df(user_animes[col].value_counts().sort("count", descending=True), f"Value Count '{col}'")


In [ ]:
# TODO build every lazy frame needed and then pl.gather
from src.schedule import Schedule

user_animes = user_animes.lazy()

schedule = Schedule.from_user_animes(user_animes)
print_df(schedule)

next_releases = user_animes.filter((pl.col("user_watch_status") == "Plan to Watch") & (pl.col("air_status") == "Not yet aired")).sort("air_start", nulls_last=True).select(
	"title",
	# "air_start",
	# "air_time",
	"air_tz",
	datetime = pl.col("air_start").dt.combine(pl.col("air_time").replace(None, time(0, 0))),
	# text = pl.format(
	# 	"{} at {} ({})",
	# 	pl.col("air_start").dt.to_string("%A %d %B %Y"),
	# 	pl.col("air_time").dt.to_string("at %H:%M"),
	# 	pl.col("air_tz"),
	# ),
).collect()
print_df(next_releases, "Next releases")